<a href="https://colab.research.google.com/github/MarceloClaro/QuantumDeepClassifier/blob/main/QuantumDeepClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.15.0
!pip install tensorflow-quantum==0.7.3
!pip install cirq
!pip install sympy
!pip install numpy
!pip install cirq[vis]


In [ ]:
# Instalar a fonte Arial
!apt-get install -y fonts-arial
# Baixar a fonte Arial
!wget -q https://befonts.com/download/arial-font -O Arial.ttf

# Criar o diretório de fontes, caso não exista
!mkdir -p ~/.fonts

# Mover a fonte para o diretório de fontes
!mv Arial.ttf ~/.fonts/

# Atualizar o cache de fontes
!fc-cache -f -v

# Atualizar o cache de fontes do Matplotlib
import matplotlib.font_manager as fm


# Definir a fonte padrão como Arial
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Arial'

import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np
from cirq.contrib.svg import SVGCircuit
# Definir a fonte padrão como DejaVu Sans
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'



In [ ]:
# Defina os qubits
qubits = [cirq.GridQubit(0, i) for i in range(2)]

# Construa o circuito quântico
circuit = cirq.Circuit()

# Adicione portas quânticas ao circuito conforme necessário
# Exemplo: Porta Hadamard no primeiro qubit
circuit.append(cirq.H(qubits[0]))

# Visualize o circuito
SVGCircuit(circuit)


In [ ]:
# Dados de entrada: 100 amostras de 2 características
train_data = np.random.rand(100, 2)
test_data = np.random.rand(20, 2)

# Rótulos binários
train_labels = np.random.randint(2, size=100)
test_labels = np.random.randint(2, size=20)


In [ ]:
def encode_data(data):
    circuits = []
    for sample in data:
        circuit = cirq.Circuit()
        for i, value in enumerate(sample):
            # Codifique o valor no ângulo de rotação do qubit
            circuit.append(cirq.rx(np.pi * value)(qubits[i]))
        circuits.append(circuit)
    return circuits

train_circuits = encode_data(train_data)
test_circuits = encode_data(test_data)


In [ ]:
train_circuit_tensors = tfq.convert_to_tensor(train_circuits)
test_circuit_tensors = tfq.convert_to_tensor(test_circuits)


In [ ]:
from tensorflow.keras import layers, initializers

layer = layers.Dense(
    units=64,
    kernel_initializer=initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=42),
    bias_initializer=initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=43)
)


In [ ]:
# Defina os parâmetros simbólicos
theta = sympy.symbols('theta0:2')

# Adicione portas parametrizadas ao circuito
for i in range(2):
    circuit.append(cirq.rx(theta[i])(qubits[i]))

# Defina a camada PQC
pqc = tfq.layers.PQC(circuit, cirq.Z(qubits[0]))

# Construa o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype=tf.string),
    pqc,
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_circuit_tensors, train_labels, epochs=10, validation_data=(test_circuit_tensors, test_labels))


In [ ]:
test_loss, test_accuracy = model.evaluate(test_circuit_tensors, test_labels)
print(f'Acurácia no teste: {test_accuracy:.4f}')


In [ ]:
import zipfile
import os
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
zip_path = '/content/melanomas.zip'
extracted_dir = '/content/melanomas'

# Extração do arquivo ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Listar os arquivos extraídos
extracted_files = os.listdir(extracted_dir)
print("Arquivos extraídos:", extracted_files)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Gerador de dados de imagem
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2  # Divisão para validação
)

# Preparar os dados de treinamento
train_data = datagen.flow_from_directory(
    extracted_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# Preparar os dados de validação
validation_data = datagen.flow_from_directory(
    extracted_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


In [ ]:
def encode_data(data):
    circuits = []
    for sample in data:
        # Achatar a imagem e normalizar
        flattened_sample = sample.flatten()
        normalized_sample = flattened_sample / np.linalg.norm(flattened_sample)

        # Criar qubits
        qubits = cirq.GridQubit.rect(1, len(normalized_sample))
        circuit = cirq.Circuit()

        # Codificar os valores nos qubits
        for i, value in enumerate(normalized_sample):
            circuit.append(cirq.rx(np.pi * value)(qubits[i]))

        circuits.append(circuit)
    return circuits

# Codificar os dados de treinamento e validação
train_circuits = encode_data(train_data[0][0])  # Primeira iteração do gerador
validation_circuits = encode_data(validation_data[0][0])


In [ ]:
# Criação de um circuito parametrizado
qubits = cirq.GridQubit.rect(1, 128)
circuit = cirq.Circuit()
theta = sympy.symbols('theta0:{}'.format(len(qubits)))
for i, qubit in enumerate(qubits):
    circuit.append(cirq.rx(theta[i])(qubit))

# Modelo híbrido
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype=tf.string),
    tfq.layers.PQC(circuit, cirq.Z(qubits[0])),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
test_loss, test_accuracy = model.evaluate(validation_circuit_tensors, validation_data[0][1])
print(f"Perda no teste: {test_loss:.4f}, Acurácia no teste: {test_accuracy:.4f}")


In [ ]:
plt.plot(history.history['accuracy'], label='Acurácia de Treinamento')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.legend()
plt.show()


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Caminho para a nova imagem
image_path = '/content/melanoma_10599.jpg'

# Carregar a imagem com o tamanho adequado
img = image.load_img(image_path, target_size=(128, 128))

# Converter a imagem para um array numpy
img_array = image.img_to_array(img)

# Normalizar os valores dos pixels para o intervalo [0, 1]
img_array = img_array / 255.0

# Achatar a imagem para um vetor
img_array = img_array.flatten()

# Adicionar uma dimensão extra para representar o lote (batch)
img_array = np.expand_dims(img_array, axis=0)


In [ ]:
import cirq
import tensorflow_quantum as tfq

def encode_image_to_circuit(image_vector):
    """Codifica um vetor de imagem em um circuito quântico."""
    # Número de qubits necessários
    num_qubits = len(image_vector)

    # Criar qubits
    qubits = cirq.GridQubit.rect(1, num_qubits)

    # Inicializar o circuito
    circuit = cirq.Circuit()

    # Codificar cada pixel como uma rotação no eixo X
    for i, pixel in enumerate(image_vector):
        circuit.append(cirq.rx(np.pi * pixel)(qubits[i]))

    return circuit

# Codificar a imagem em um circuito
image_circuit = encode_image_to_circuit(img_array[0])

# Converter o circuito para um tensor compatível com o TensorFlow
image_circuit_tensor = tfq.convert_to_tensor([image_circuit])


In [ ]:
# Realizar a predição
prediction = model.predict(image_circuit_tensor)

# Exibir a predição
print(f'Predição bruta: {prediction[0][0]}')


In [ ]:
# Definir um limiar para classificação binária
threshold = 0.5

# Determinar a classe prevista
predicted_class = 1 if prediction[0][0] >= threshold else 0

# Mapear a classe prevista para o rótulo correspondente
class_labels = {0: 'Normal', 1: 'Melanoma'}
predicted_label = class_labels[predicted_class]

print(f'A imagem foi classificada como: {predicted_label}')


In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit


In [ ]:
import zipfile
import os

# Caminho para o arquivo ZIP
zip_path = '/content/melanomas.zip'

# Diretório de extração
extracted_dir = '/content/melanomas'

# Extração do arquivo ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Verificação dos arquivos extraídos
extracted_files = os.listdir(extracted_dir)
print(f'Arquivos extraídos: {extracted_files}')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Gerador de dados com pré-processamento
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Parâmetros
img_height, img_width = 128, 128
batch_size = 32

# Dados de treinamento
train_data = datagen.flow_from_directory(
    extracted_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

# Dados de validação
val_data = datagen.flow_from_directory(
    extracted_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


In [ ]:
def encode_image_to_circuit(image):
    """Codifica uma imagem em um circuito quântico."""
    qubits = cirq.GridQubit.rect(1, img_height * img_width)
    circuit = cirq.Circuit()
    for i, pixel in enumerate(image.flatten()):
        if pixel > 0:
            circuit.append(cirq.rx(np.pi * pixel)(qubits[i]))
    return circuit


In [ ]:
train_circuits = [encode_image_to_circuit(img) for img, _ in train_data]
val_circuits = [encode_image_to_circuit(img) for img, _ in val_data]

train_labels = np.array([label for _, label in train_data])
val_labels = np.array([label for _, label in val_data])

train_circuit_tensors = tfq.convert_to_tensor(train_circuits)
val_circuit_tensors = tfq.convert_to_tensor(val_circuits)


In [ ]:
# Converta os circuitos quânticos para tensores
train_circuits = tfq.convert_to_tensor([convert_image_to_circuit(img) for img, _ in train_generator])
validation_circuits = tfq.convert_to_tensor([convert_image_to_circuit(img) for img, _ in validation_generator])

# Extraia os rótulos
train_labels = np.array([label for _, label in train_generator])
validation_labels = np.array([label for _, label in validation_generator])

# Treine o modelo
history = model.fit(
    train_circuits, train_labels,
    validation_data=(validation_circuits, validation_labels),
    epochs=10,
    batch_size=batch_size
)


In [ ]:
# Definição dos qubits
qubits = cirq.GridQubit.rect(1, img_height * img_width)

# Função para criar o modelo QCNN
def create_qcnn_model():
    model_circuit = cirq.Circuit()
    # Adicione as portas quânticas conforme a arquitetura desejada
    # ...

    readout_operators = [cirq.Z(qubit) for qubit in qubits]

    qcnn = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(), dtype=tf.dtypes.string),
        tfq.layers.PQC(model_circuit, readout_operators),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    return qcnn

model = create_qcnn_model()


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_circuit_tensors, train_labels,
    validation_data=(val_circuit_tensors, val_labels),
    epochs=10,
    batch_size=batch_size
)


In [ ]:
# Avaliação
loss, accuracy = model.evaluate(val_circuit_tensors, val_labels)
print(f'Acurácia: {accuracy * 100:.2f}%')

# Classificação de uma nova imagem
new_image_path = '/content/melanoma_10599.jpg'
# Carregue e pré-processe a imagem conforme os passos anteriores
# ...
# Codifique a imagem em um circuito quântico
new_circuit = encode_image_to_circuit(new_image)
new_circuit_tensor = tfq.convert_to_tensor([new_circuit])

# Predição
prediction = model.predict(new_circuit_tensor)
print(f'Predição: {prediction[0][0]}')
